In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn

batch_size = 4
max_length = 3
hidden_size = 2
n_layers = 1
input_dim = 1
batch_first = False

# Data
vec_1 = torch.FloatTensor([[1, 2, 3]]).view(3, -1)
vec_2 = torch.FloatTensor([[1, 2, 0]]).view(3, -1)
vec_3 = torch.FloatTensor([[1, 0, 0]]).view(3, -1)
vec_4 = torch.FloatTensor([[2, 0, 0]]).view(3, -1)

# Put the data into a tensor.
batch_in = torch.zeros((max_length, batch_size, input_dim))
batch_in[:, 0, : ] = vec_1
batch_in[:, 1, :] = vec_2
batch_in[:, 2, :] = vec_3
batch_in[:, 3, :] = vec_4

# Wrap RNN input in a Variable. Shape: (batch_size, max_length, input_dim)
batch_in = Variable(batch_in)
# The lengths of each example in the batch. Padding is 0.
seq_lengths = torch.LongTensor([3, 2, 1, 1])

# Wrap input in packed sequence, with batch_first=True
packed_input = torch.nn.utils.rnn.pack_padded_sequence(
    batch_in, seq_lengths, batch_first=batch_first)

# Create an RNN object, set batch_first=True
rnn = nn.RNN(input_dim, hidden_size, n_layers, batch_first=batch_first) 

# Run input through RNN 
packed_output, _ = rnn(packed_input)

# Unpack, with batch_first=True.
output, _ = torch.nn.utils.rnn.pad_packed_sequence(
    packed_output, batch_first=batch_first)
print("Unpacked, padded output: ")
print(output)

# Extract the outputs for the last timestep of each example
print('seq_lengths: ', seq_lengths)
idx = (torch.LongTensor(seq_lengths) - 1).view(-1, 1).expand(
    len(seq_lengths), output.size(2))

time_dimension = 1 if batch_first else 0
idx = idx.unsqueeze(time_dimension)

if output.is_cuda:
    idx = idx.cuda(output.data.get_device())
    
# Shape: (batch_size, rnn_hidden_dim)
last_output = output.gather(
    time_dimension, Variable(idx)).squeeze(time_dimension)
print("Last output: ")
print(last_output, last_output.size())

Unpacked, padded output: 
tensor([[[0.6311, 0.5253],
         [0.6311, 0.5253],
         [0.6311, 0.5253],
         [0.6268, 0.7628]],

        [[0.5639, 0.3431],
         [0.5639, 0.3431],
         [0.0000, 0.0000],
         [0.0000, 0.0000]],

        [[0.5598, 0.7307],
         [0.0000, 0.0000],
         [0.0000, 0.0000],
         [0.0000, 0.0000]]], grad_fn=<CopySlices>)
seq_lengths:  tensor([3, 2, 1, 1])
Last output: 
tensor([[0.5598, 0.7307],
        [0.5639, 0.3431],
        [0.6311, 0.5253],
        [0.6268, 0.7628]], grad_fn=<SqueezeBackward1>) torch.Size([4, 2])
